# Straka Bubble Test 🫧🫧🫧

<div class="alert alert-block alert-success">

## Part 3: Generating the executable $\texttt{PADDLE}$ code

Now that we have completely set up the physics (**Part 1**) and simulation properties (**Part 2**) Straka problem from scratch, we can now actually run the simulation! 

In general, with $\texttt{PADDLE}$, users will rarely have to generate their own `.py` or `.ipynb` scripts from scratch to run simulations based on examples covered in tutorial notebooks. 

However, more advanced users might want to develop their own scripts, so we provide detailed comments on the script below. 

$\texttt{PADDLE}$ is a python-wrapper for codes like [$\texttt{kintera}$](https://github.com/chengcli/kintera) and [$\texttt{snapy}$](https://github.com/chengcli/snapy), so for actual details for the thermodynamics and dynamical codes, we will refer users to explore those codes.  

As done when generating the `.yaml` file, we recommend having the `straka.py` script, found in folder, open while we go through the next few cells below. 

For those wanting to skip ahead to dropping the bubble, just run the next few cells! 

</div>

<div class="alert alert-info">

## Straka Bubble Free Parameters

In **Part 1**, we defined a variety of free parameters that are specific to the physics of our problem. In particular, we defined 

1. The surface temperature (*T_s*) and pressure (*P_s*) of the background atmosphere 
2. The horizontal coordinate for the center of the bubble (*xc*) and the horizontal extent (*xr*)
3. The vertical coordinate for the center of the bubble (*zc*) and the vertical extent (*zr*)
4. The maximum temperature anomaly of the bubble (*dT*)

Here we set them explicitly, as these are **not** part of the `.yaml` file. 

In [1]:
# Define the problem's free parameters

# Ps is the surface pressure
# 1 bar = 1e5 pascals 
Ps = 1e5 # in pascals

# surface temperature 
Ts = 300 # in K 

# Horizontal coordinate for center of bubble 
xc = 0 # in m 

# Horizontal extent of bubble
xr = 4e3 # in m

# Vertical coordinate for center of bubble
zc = 3e3 # in m

# Vertical extent of bubble
zr = 2e3 # in m

# Maximum temperature anomaly 
dT = -15 

<div class="alert alert-info">

## Tracking Primitive Variables in $\texttt{PADDLE}$ 

As defined in **Part 1**, in dynamical simulations (specifically within compressible, hydrodynamic codes), **primitive variables** are fundamental physical quantities used to solve the state of the system via the primitive equations (the equations that approximate atmospheric flows). 

These variables are important since they are what sets the initial condition of the simulation, and are what are looked at as default outputs (in the $\texttt{outputs}$ dictionary of the `.yaml` file, we explicitly set `prim` as an output).

The five primitive variables used in $\texttt{PADDLE}$ are: density, velocity (one for each dimension), and pressure. 

In $\texttt{PADDLE}$ the primitive variables are stored in a 4-dimension array (called a tensor) with a specific order: 

1. density (rho, index = 0)
2. velocity of dimension 1 (vel1, vertical velocity, index = 1)
3. velocity of dimension 2 (vel2, horizontal [width] velocity, index = 2)
4. velocity of dimension 3 (vel3, horizontal [depth] velocity, index = 3)
5. pressure (press, index = 4)

In order to make the code below more readable, $\texttt{PADDLE}$ uses structured names to refer to each primitive variable's index (where $\texttt{pytorch}$'s naming convention is used for **k**, which refers to a *constant*. **I** is $\texttt{athena++}$'s naming convention for *index of*): 

1. kIDN = 0 (density)
2. kIV1 = 1 (vel1)
3. kIV2 = 2 (vel2)
4. kIV3 = 3 (vel3)

<div>

First, import the necessary packages.

In [2]:
# Torch is the basic computational backend, equivalent in some ways to numpy but can work on CPU and GPU
import torch

# Basic package for mathematical functions
import math

# kintera is a package developed by the same developers as PADDLE and CANOE that performs thermodynamic calculations 
# see https://github.com/chengcli/kintera
import kintera

# snapy is a package developed by the same developers as PADDLE and CANOE that performs dynamical calculations 
# see https://github.com/chengcli/snapy

# Here we import the main module (MeshBlock) and options (MeshBlockOptions) used in all PADDLE simulations (explained further below)
# and the named locations of density and pressure (which we use below to define our initial condition)
from snapy import MeshBlockOptions, MeshBlock
from snapy import kIDN, kIPR

# os is used to create the output directory for outputs 
import os 

<div class="alert alert-warning">

### Conserved Variables, Diagnostic Variables, and User Output Variables

As discussed in **Part 1**, ither variables of interest in a dynamical code are **conserved variables** and **diagnostic variables**. 

**Conserved variables** are those that remain constant when an integral of the entire simulation domain is taken at each time step. These include momentum, energy, and mass. While these are not typically used as outputs, they are essential when solving the dynamical simulation. 

**Diagnostic variables** are variables that are not used explicitly by the dynamical simulation, but can be computed from either the **primitive** or **conserved variables**. The most common are temperature and potential temperature, which can be computed as an output using the equation of state and the primitive variables.

Since we are interested in temperature and potential temperature, we can manually include them in our output. 

<div>

<div class="alert alert-info">

## Creating custom user output variables 

As mentioned in the $\texttt{outputs}$ section in **Part 2**, the output files can include user output variables (**uov**). 

These user output variables are manually defined in a user-defined function called **call_user_output()**. 

Here we want temperature and potential temperature. As in **Part 1**, from the ideal gas law temperature is given by  

$T = \frac{P}{R_d \rho}$

and potential temperature by 

$\theta = T(\frac{P_s}{P})^{\frac{R}{c_p}}$

In [3]:
# User defined function to generate user output variables (uov)

# Input : bvars (block_variables, stores all the variables in the simulation)
# we will generating the block variables object in the following cells to this one

# Output: dictionary with user output variables

def call_user_output(bvars):

    # Load in the primitive variables stored by the simulation 
    # As we will see in more detail in the cells below, hydro_w is a key that signifies the primitive variables 
    # where it stores each of the primitive variable values at each cell in the simulation 
    hydro_w = bvars["hydro_w"]

    # Create the user output variable dictionary 
    out = {}

    # Manually compute temperature and potential temperature 
    # from primitive variables and equation of state properties

    # kIPR = Index of Pressure 
    # kIDN = Index of Density  

    # Rd = specific gas constant
    # Ps = surface pressure
    # cp = specific heat capacity at const pressure 
    
    # Temperature 
    temp = hydro_w[kIPR] / (Rd * hydro_w[kIDN])

    # Define it in output dictionary 
    out["temp"] = temp
    
    # Potential Temperature 
    theta = temp * (Ps / hydro_w[kIPR]).pow(Rd / cp)

    # Define it in output dictionary 
    out["theta"] = theta

    # Output the user output variable dictionary 
    return out

<div class="alert alert-info">

## Generate the $\texttt{MeshBlock}$ Object

In $\texttt{PADDLE}$, the main object used to store all the information of the 3D hydrodynamic simulation is called the `MeshBlock` module. We will need to create this module so that we can generate our initial condition of the simulation. 

We must first initialize this object from our `.yaml` configuration file before we set up our simulation's initial condition. 

This module stores all the **operations** (i.e., functions that are evolved in a simulation) that is performed on a 3D grid. 

This object is a heavy object that has a complex internal data structure. Therefore, to construct the object we use an intermediary object called the `MeshBlockOptions` object. 

Lets go step-by-step and create the `MeshBlock` object.


<div>

First, we generate an **options** object from our `yaml` configuration file we generated in **Part 2**. We also create and set the output directory for all the output files (without an output directory set, all output files are saved in the same directory as the script below is run in)

In [4]:
# set hydrodynamic options from our yaml file
op = MeshBlockOptions.from_yaml("./straka_made_in_notebook.yaml")

# Or, use the default one by commenting out this line of code 
# op = MeshBlockOptions.from_yaml("./straka.yaml")

# Set output directory 
output_directory = './straka_results/'
os.makedirs(output_directory, exist_ok=True)
op.output_dir(output_directory)

[HydroOptions] -- constant gravity options --
* grav1 = -9.8
* grav2 = 0
* grav3 = 0


MeshBlockOptions(
-- meshblock options --
* verbose = false
* basename = straka_made_in_notebook
)

When we generate the `MeshBlockOptions` object, diagnostic information is printed out to show that our gravity options have been correctly registered in the object. 

Next, we can construct the `MeshBlock` object from the `MeshBlockOptions` object. 

In [5]:
# initialize block 
block = MeshBlock(op)

[W219 15:31:02.285059000 ProcessGroupGloo.cpp:757] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Once we have the `MeshBlock` object initialized, we can now access essential submodules needed to define our initial condition of the simulation. 

Both the `MeshBlock` object and `MeshBlockOptions` object have a nested submodule structure 

As a user of $\texttt{PADDLE}$, you do not need to know the full structure of these objects, as all example tutorials will provide example code on which submodules and sub-options to access for specific problems, and we include the below just to demonstrate snippets of how the objects are structured: 

    ├── MeshBlock
    │   ├── Coordinate (coord)
    |   ├── Hydro (hydro)
    |   │   ├── EquationOfState (eos)
    |   │   ├── ...
    │   ├── ...

    ├── MeshBlockOptions
    │   ├── CoordinateOptions (coord)
    |   ├── HydroOptions (hydro)
    |   │   ├── EquationOfStateOptions (eos)
    |   |   │   ├── weight()
    |   |   │   ├── ...
    |   │   ├── ConstGravityOptions (grav)
    |   |   │   ├── grav1()
    |   |   │   ├── grav2()
    |   |   │   ├── grav3()
    |   │   ├── ...
    |   │   ├── ...
    │   ├── ...


`MeshBlock` module properties are accessed using `block.module()`, while options are accessed using `block.options()`. 

In [6]:
# get handles to specific modules

# Coordinates of simulation domain stored in MeshBlock
# Accessed via .module('name-of-submodule')
# We will need this to generate the meshgrid we will define our initial condition of the simulation on top of 
coord = block.module("coord")

# Equation of state properties stored in MeshBlock
# Accessed via .module('name_of_submodule.name_of_sub_submodule')
# We will need this to generate our thermodynamic constants used when defining our initial condition 
eos = block.module("hydro.eos")

# Vertical gravity stored in MeshBlock (initialized by MeshBlockOptions)
# Options are accessed differently than modules 
# Accessed via .options.sub_module.sub_submodule.sub_sub_submodule() [chaining together module names with '.']
# We will need this to generate our initial temperature condition (see Part 1, dT/dz)
grav = -block.options.hydro().grav().grav1()

From **Part 1**, we know that we will also additionally need two more variables to fully define our initial conditions: the specific heat constant, Rd, and specific heat capacity at constant pressure, cp. 

In [7]:
# thermodynamic constants 

# Specific gas constant
# We defined the molecular weight (M) in the dynamics dictionary of the .yaml file 
# Used in call_user_output to get temperature and potential temperature
# Used to set initial condition of temperature and pressure
Rd = kintera.constants.Rgas / eos.options.weight()

# Specific heat capacity at constant volume
# Computed from defined weight (M) and adiabatic index (gamma) in dynamics dictionary 
# In an ideal gas, cv = R / gamma - 1
# Used to get cp
cv = eos.species_cv_ref()

# Specific heat capacity at constant pressure 
# Used in call_user_output to get temperature and potential temperature
# Used to set initial condition of temperature and pressure
cp = cv + Rd

<div class="alert alert-info">

## Creating the Initial Condition

Now, we want to actually define our simulation's initial condition. 

In order to do this, we follow a similar flow as we did in **Part 1**, where we create our 3-dimensional grid and then define the primitive variables at each point in the grid. 

<div>

We first want to explicitly access the coordinates of the simulation grid in order to set our initial condition. 

We do this by generating a tensor with **pytorch** (similar to a numpy array, except it can work with both CPUs and GPUs).

To create the tensor, we access the `buffer` of the `coord` object defined above. A `buffer` is just the actual data owned by a specific submodule of `MeshBlock`. 

The buffer data owned by the `Coordinate` module is defined at cell centers and interfaces. Here, we will use the centers to create the meshgrid:

    ├── Coordinate
    │   ├── "x1v" (vertical coordinate at cell center)
    |   ├── "x1f" (vertical coordinate at cell interface)
    │   ├── "x2v" (horizontal coordinate at cell center)
    │   ├── "x2f" (horizontal coordinate at cell interface)
    │   ├── "x3v" (horizontal coordinate at cell center)
    │   ├── "x3f" (horizontal coordinate at cell interface)

In [8]:
# Setup the meshgrid for the simulation

# coord.buffer("x1v") 
# returns a 1D tensor with length = number of cells in x1 direction + 2 x ghost zoness
# As defined in the .yaml file, this would be 64 cells + 2 x 3 ghost zones = 70

x3v, x2v, x1v = torch.meshgrid(
    coord.buffer("x3v"), coord.buffer("x2v"), coord.buffer("x1v"), indexing="ij"
)

# The number of points in each dimension
# This is identical to the number of cells 
nc3 = coord.buffer("x3v").shape[0]
nc2 = coord.buffer("x2v").shape[0]
nc1 = coord.buffer("x1v").shape[0]

Now we want to set up the initial condition of the problem. 

First, we hardcode the number of variables stored at each coordinate to 5 (with **nvar**). 

This represents the five primitive variables (density, vel1, vel2, vel3, and pressure). We can hardcode the number of variables here since we are assuming an ideal gas equation of state (but we note that in future tutorials, with more sophisticated equations of states, this will not always be the case). 

Then we set our initial condition. **The initial condition of any simulation in $\texttt{PADDLE}$ will always be defined in terms of primitive variables.** This is usually the most difficult part of the problem as you will need to figure out the initial physics (as we did in **Part 1**) and need to make sure the initial background atmosphere is hydrostatically balanced. 

Lucky for us, in **Part 1** we discovered that by using the ideal gas equation-of-state with a constant heat capacity, there is an analytic solution for density and pressure that simplifies things (in future tutorials, we will cover how to get initial conditions for more sophisticated equations of states).

In [9]:
# Hardcode the number of variables (5 = number of primitive variables)
nvar = 5

# Define a 4D tensor storing primitive variables at each simulation coordinate
# In athena++, w signifies the primitive variables 
w = torch.zeros((nvar, nc3, nc2, nc1), device='cpu')

# Define the spatial extent of the bubble as we did in Part 1
L = torch.sqrt(((x2v - xc) / xr) ** 2 + ((x1v - zc) / zr) ** 2)

# Set the temperature field of the entire simulation 
# As we did in Part 1, see dT/dz
temp = Ts - grav * x1v / cp

# Initial Condition of the full Atmosphere's Pressure Field, defined as done in Part 1
# Recall we define pressure before applying the bubble's temperature anomaly
# since pressure equilibrates at the speed of sound
w[kIPR] = Ps * torch.pow(temp / Ts, cp / Rd)

# Add the temperature anomaly of the bubble to the temperature field
temp += torch.where(L <= 1, dT * (torch.cos(L * math.pi) + 1.0) / 2.0, 0)

# Initial Condition of the full Atmosphere's Density Field, defined as done in Part 1
w[kIDN] = w[kIPR] / (Rd * temp)

Next, we actually use the conditions above to set the initial condition. 

We first create the `block_vars` dictionary that holds the initial condition tensors. In this case, we set the initial `hydro_w`, which represents the hydrodynamical primitive variables. 

Then we call `block.initialize()`, which initializes the `MeshBlock` object with the initial `hydro_w` variables and automatically computes the conserved variables (`hydro_u`). This function returns both an expanded dictionary of tensors (with the primitive **and** conserved variables) that will be updated at each cycle of the simulation's integration, and the current simulation time. 

Finally, we let the code know that we set up a custom `call_user_output()` function in order to output the temperature and primitive temperature alongside the primitive variables. 

In [10]:
# Create an empty block variables dictionary
block_vars = {}

# Populate the primitive variables with the initial condition we created above
block_vars["hydro_w"] = w

# Initialize the MeshBlock (block) object with the block variables 
block_vars, current_time = block.initialize(block_vars)

# Let the simulation know we created a custom call_user_output function 
# to output temperature and potential_temperature
block.set_user_output_func(call_user_output)

<div class="alert alert-info">

## Run the simulation!

Below, we actually start integrating the model.

The code below will be more-or-less the same for all $\texttt{PADDLE}$ python scripts, and can be taken as-is for now. 

In future tutorials, we will notate when specific code (like external forcing) is added to the loop below. 

<div>

In [11]:
# Initialize the output before the simulation starts, compute until the desired time 
block.make_outputs(block_vars, current_time)

# Run the simulation in a loop
while not block.intg.stop(block.inc_cycle(), current_time):

    # Each time step of the simulation is determined by the cfl number and the sound speed 
    dt = block.max_time_step(block_vars)

    # Output to let us know the code is running
    block.print_cycle_info(block_vars, current_time, dt)

    # For each cycle, a multi-stage method (here rk3) is used to advance block_vars
    for stage in range(len(block.intg.stages)):
        block.forward(block_vars, dt, stage)

    # Check for any errors
    err = block.check_redo(block_vars)
    if err > 0:
        continue  # redo current step
    if err < 0:
        break  # terminate

    # Progress the time and make outputs 
    current_time += dt
    block.make_outputs(block_vars, current_time)

# Make the final outputs and clean up the internal states in MeshBlock
block.finalize(block_vars, current_time)

cycle=10 redo=0 time=2.34819714317398e+00 dt=2.60910784041928e-01 mass0=3.76823726405015e+12 energy=7.24829775220410e+17
cycle=20 redo=0 time=4.95726502936207e+00 dt=2.60888806614001e-01 mass0=3.76823726405014e+12 energy=7.24834317084604e+17
cycle=30 redo=0 time=7.56568333145982e+00 dt=2.60751713246774e-01 mass0=3.76823726405014e+12 energy=7.24841931702870e+17
cycle=40 redo=0 time=1.01718638112876e+01 dt=2.60433946220395e-01 mass0=3.76823726405014e+12 energy=7.24851468226832e+17
cycle=50 redo=0 time=1.27746701548911e+01 dt=2.60111781930019e-01 mass0=3.76823726405014e+12 energy=7.24860609991423e+17
cycle=60 redo=0 time=1.53748349800135e+01 dt=2.59924713653201e-01 mass0=3.76823726405014e+12 energy=7.24867389729864e+17
cycle=70 redo=0 time=1.79736258305092e+01 dt=2.59833466991663e-01 mass0=3.76823726405014e+12 energy=7.24871128964159e+17
cycle=80 redo=0 time=2.05716489786142e+01 dt=2.59761090090646e-01 mass0=3.76823726405014e+12 energy=7.24871774882728e+17
cycle=90 redo=0 time=2.316881467

<div class="alert alert-block alert-success">

The cell above should have printed out a myriad of statements every 10 cycles of the simulation (as defined by `ncycle_out` in the `.yaml` file), and ran until 900 seconds (as defined by `tlim` in the `.yaml` file) has passed in the simulation. 

Additionally, the output directory (`straka_results`) will have been populated by the following files: 

1. testing.0000X.restart (X ranging from 0 to 3), and testing.final.restart
2. testing.out1.XXXXX.nc (XXXXXX ranging from 00000-00030)

On we go to **Part 4**, where we will analyze our outputs.